### Subset for all pages from 1880s (est & liv goub vedomosti)

In [3]:
library(tidyverse)

In [4]:
library(tidytext, lib.loc = "~/R_pckg/")

In [5]:
dorevol <- read.csv("test/dorevol.csv")
subset_meta_pages <- read.delim("test/meta_rus_subset_pages.tsv", sep = "\t")
test_join <- inner_join(dorevol, subset_meta_pages, by = "PageID")
str(test_join)

'data.frame':	128700 obs. of  20 variables:
 $ X                         : int  1 2 3 4 5 6 7 8 9 10 ...
 $ PageID                    : chr  "borbaorgan19171208.1.4" "borbaorgan19171217.1.4" "borbaorgan19171201.1.1" "borbaorgan19171201.1.2" ...
 $ text                      : chr  "№ 4 „Борьба* 1 4  ныхъ случаяхъ Нрезщцумъ можетъ давать распоряжены по своему усмотр’Ьнш и во всехъ своихъ дейс"| __truncated__ "Но 8 Борьба\"\". 4  ТЕЛЕГРАММЫ. ВсЪмъ народамъ и правительствамъ союзныхъ странъ. Мирные переговоры ведущееся в"| __truncated__ "/#&lt;г  Вт» борьб-Ь обретешь ты право свое! ^  Ц’Ьна отд. № 15 коп. Адресь редакцш: Ревель, Гонзюрская у.д.№4,"| __truncated__ "ЛП I „Борьб а“. 2  отъ жизни и массъ, проводить какой то дьявольшй саботаже. тормозя работу и только и мечтаетъ"| __truncated__ ...
 $ .id                       : int  28 28 28 28 28 28 28 28 28 28 ...
 $ docid                     : chr  "borbaorgan19171208" "borbaorgan19171217" "borbaorgan19171201" "borbaorgan19171201" ...
 $ at

In [9]:
np_1880 <- test_join %>% filter(year > 1879 & year < 1891) %>% 
    mutate(id_per = str_replace_all(docid, "(\\w+)(\\d{8})", "\\1"))
unique(np_1880$id_per)

[1] "ekmteataja" "livzeitung"

In [10]:
dorevol_langs <- read.csv("test/dorevol_langs.csv")
str(dorevol_langs)

'data.frame':	128700 obs. of  3 variables:
 $ X     : int  1 2 3 4 5 6 7 8 9 10 ...
 $ PageID: chr  "borbaorgan19171208.1.4" "borbaorgan19171217.1.4" "borbaorgan19171201.1.1" "borbaorgan19171201.1.2" ...
 $ lang  : chr  "russian-koi8_r" "russian-koi8_r" "russian-koi8_r" "russian-koi8_r" ...


In [11]:
np_1880_langs <- inner_join(np_1880, dorevol_langs, by = "PageID")
np_1880_langs <- np_1880_langs %>% 
    mutate(lang_cln = str_replace_all(lang, "^(\\w+)-.*", "\\1")) %>% 
    filter(lang_cln == "german" | lang_cln == "russian")

table(np_1880_langs$lang_cln)


 german russian 
   6346    8717 

In [12]:
np_1880_langs <- np_1880_langs %>% 
    group_by(lang_cln) %>% 
    slice_sample(n = 3000) %>% 
    ungroup()
str(np_1880_langs)

tibble [6,000 × 24] (S3: tbl_df/tbl/data.frame)
 $ X.x                       : int [1:6000] 22221 69718 71038 69596 67814 70561 68880 23392 23647 70463 ...
 $ PageID                    : chr [1:6000] "ekmteataja18811212.1.1" "livzeitung18820702.1.3" "livzeitung18830927.1.5" "livzeitung18820602.1.1" ...
 $ text                      : chr [1:6000] "Эстляндские губернские ведомости  Эстляндсш Губернсшя Ведомости выходятъ по вторникамъ, четвергаиъ и субботамъ."| __truncated__ "Siimmlllche Polizeibehörden beehrt sich das Rigasche Ordnungsgericht hierdurch zu ersuchen, den zu Friedrichsta"| __truncated__ "Kaukegesindes anöden Jahn Spohr, von dem Peter Stabit und dem Carl Stabit unterzeichnet werde und nunmehr Ehrma"| __truncated__ "Лифляндские губернские ведомости  Livländische Gouvernements-Zeitung  Годъ XXX.  Губериск{я Ведомости выходятъ "| __truncated__ ...
 $ .id                       : int [1:6000] 43 77 77 77 77 77 77 43 43 77 ...
 $ docid                     : chr [1:6000] "ekmteataj

In [14]:
table(np_1880_langs$lang_cln)


 german russian 
   3000    3000 

#### Tokenization

In [15]:
start_time <- Sys.time()
tokens <- np_1880_langs %>%
    unnest_tokens(input = text, output = word, token = "words")

top_tokens_1880 <- tokens %>% 
    group_by(PageID, word) %>% 
    count(sort = T) %>%
    ungroup()

end_time <- Sys.time()

time_taken <- round(end_time - start_time,2)
time_taken

Time difference of 6.2 mins

In [16]:
head(top_tokens_1880)

PageID,word,n
<chr>,<chr>,<int>
livzeitung18800709.1.9,500,260
livzeitung18820531.1.9,gr,260
livzeitung18820113.1.1,500,259
livzeitung18820531.1.9,groß,259
livzeitung18820531.1.9,nr,259
livzeitung18820915.1.1,500,258


In [17]:
top_tokens_1880 <- top_tokens_1880 %>% 
    filter(!str_detect(word, "^\\d+$|^\\W+$")) %>% 
    filter(!str_detect(word, "^\\w$|^\\w\\w$"))
head(top_tokens_1880)

PageID,word,n
<chr>,<chr>,<int>
livzeitung18820531.1.9,groß,259
livzeitung18820531.1.9,thlr,257
livzeitung18871113.1.6,лес,165
livzeitung18851023.1.2,der,157
livzeitung18800611.1.6,groß,152
livzeitung18800613.1.5,groß,152


In [19]:
write.csv(top_tokens_1880, "mfw/biling_1880.csv")